# Technical Factor - Current Ration (CR)

In [5]:
import dai
import pandas as pd

In [6]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [7]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        (2 * close + high + low) / 4 AS MID,
        m_lag(MID, 1) AS MID_L1,
        m_sum(high - MID_L1, 26) / m_sum(MID_L1 - low, 26) AS CR,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(CR > 100 AND m_lag(CR, 1) <= 100, 1, 0) AS TRBY1,
        IF(CR < 100 AND m_lag(CR, 1) >= 100, 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(CR > 100 AND (CR - m_lag(CR, 1)) > 0, 1, 0) AS MTBY1,
        IF(CR < 100 AND (CR - m_lag(CR, 1)) < 0, 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(CR < 70 AND (CR - m_lag(CR, 1)) > 0, 1, 0) AS RVBY1,
        IF(CR > 130 AND (CR - m_lag(CR, 1)) < 0, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(CR, 1) <= 150 AND CR > 150, 1, 0) AS BKBY1,
        IF(m_lag(CR, 1) >= 50 AND CR < 50, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        MID,
        CR,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [8]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,MID,CR,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-02-17,600519.SH,12026.052156,1.011007,0,0,0,0,0,0,0,0
1,2025-02-18,600519.SH,12025.543037,1.108382,0,0,0,0,1,0,0,0
2,2025-02-19,600519.SH,12099.406081,1.359709,0,0,0,0,1,0,0,0
3,2025-02-20,600519.SH,12042.425436,1.321672,0,0,0,1,0,0,0,0
4,2025-02-21,600519.SH,12109.221903,1.406715,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
212,2025-12-25,600519.SH,11929.686755,0.800288,0,0,0,0,1,0,0,0
213,2025-12-26,600519.SH,11947.003088,0.821241,0,0,0,0,1,0,0,0
214,2025-12-29,600519.SH,11866.186493,0.771665,0,0,0,1,0,0,0,0
215,2025-12-30,600519.SH,11756.819064,0.729303,0,0,0,1,0,0,0,0
